In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df_stations = pd.read_json("../Andres/stations.json")

In [4]:
for k  in df_stations["list"][0].keys():
    
    df_stations[k] = df_stations["list"].apply( lambda x : x[k] )
df_stations = df_stations.drop( ["list"] , axis = 1 )

#df_stations = df_stations[ df_stations["description"] != "Hakaniemen kauppahalli 1" ]
serial2name = dict()
name2pos = dict( )
for x , y  in df_stations.iterrows():
    serial2name[ y["serial"]] = y["description"]
    name2pos[ y["description"] ] = ( y["latitude"] , y["longitude"] )
serials = [k for k, v in serial2name.items()]
serials

['0000000006dd41f6',
 '000000000f4919c9',
 '0000000019fb59c4',
 '00000000342570c2',
 '0000000038bf9618',
 '0000000038d83d41',
 '000000004ef3150d',
 '0000000053c6c2be',
 '00000000675d5200',
 '000000006b087f40',
 '000000006b44cce7',
 '0000000074f765a7',
 '000000007b5207b6',
 '0000000096918cfa',
 '00000000a53ed894',
 '00000000aa852af1',
 '00000000afef4555',
 '00000000b33a8357',
 '00000000cd16b8ef',
 '00000000d747f075',
 '00000000e8a064a4',
 '00000000f1124bca',
 '00000000fb7600be',
 '00000000fdda10fe',
 '00000000fffb8cf0']

In [ ]:
adj_mat = pd.DataFrame(data = 0, index=serials, columns = serials)
num_days = 63

for i in range(num_days):
    print("Day " + str(i) + " is being computed!")
    df = pd.read_pickle("raw_hypercell_20-21_" + str(i))
    df = df.drop(columns = ["distance", "time"])
    df = df.drop_duplicates()
    df = df.groupby("hash", as_index = False).agg(list)
    df["count"] = df["serial"].apply(len)
    df = df[df["count"] > 1]
    df.reset_index(inplace=True, drop=True)

    for i in range(len(df)):
        row = df.loc[i]
        if row["count"] > 2:
            df.drop(index=i, inplace=True)
            l = row["serial"]
            pairs = list(zip(l[::], l[1::]))
            for pair in pairs:
                new_row = pd.DataFrame(data = {"hash" :row["hash"], "serial" : [list(pair)], "count" : 2 })
                df = df.append(new_row, sort=True)

    df.reset_index(inplace=True, drop=True)
    
    for i, x in df.iterrows():
        start_station = x["serial"][0]
        stop_station = x["serial"][1]
        adj_mat.at[str(start_station), str(stop_station)] = adj_mat[str(start_station)][str(stop_station)] + 1
    
adj_mat = adj_mat/num_days

Day 0 is being computed!
Day 1 is being computed!
Day 2 is being computed!
Day 3 is being computed!
Day 4 is being computed!
Day 5 is being computed!
Day 6 is being computed!
Day 7 is being computed!
Day 8 is being computed!
Day 9 is being computed!
Day 10 is being computed!
Day 11 is being computed!
Day 12 is being computed!
Day 13 is being computed!
Day 14 is being computed!
Day 15 is being computed!
Day 16 is being computed!
Day 17 is being computed!
Day 18 is being computed!
Day 19 is being computed!
